In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\brain\\TUMOR-DETECTION-MLOPS-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\brain\\TUMOR-DETECTION-MLOPS-'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class FeatureConfig:
    root_dir: Path
    brain_df: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_feature_engineering_config(self) -> FeatureConfig:
        config = self.config.feature_engineering
        
        create_directories([config.root_dir])

        feature_eng_config = FeatureConfig(
            root_dir=Path(config.root_dir),
            brain_df=Path(config.brain_df),
          
        )

        return  feature_eng_config


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
class FeatureEngineering:
    def __init__(self, config:FeatureConfig):
        self.config = config
        self.brain_df = None
        self.brain_df_train = None
        self.train = None
        self.test = None


    def preprocess_data(self):
        brain_df = pd.read_csv(self.config.brain_df)
        brain_df_train = brain_df.drop(columns=['patient_id'])
        brain_df_train['mask'] = brain_df_train['mask'].apply(lambda x: str(x))
        train,test = train_test_split(brain_df_train, test_size=0.15)

        train_file_path = os.path.join("artifacts/data_ingestion/Brain_MRI", "train.csv")
        test_file_path = os.path.join("artifacts/data_ingestion/Brain_MRI", "test.csv")

        train.to_csv(train_file_path, index=False)
        test.to_csv(test_file_path, index=False)

        print("The training and testing CSV files have been saved.")


# Usage
# data_processor = DataProcessor('data_mask.csv')
# data_processor.load_data()
# data_processor.preprocess_data()
# data_processor.split_data()

In [25]:
try:
    config = ConfigurationManager()
    feature_engineering_config = config.get_feature_engineering_config()
    feature_engineering = FeatureEngineering(config=feature_engineering_config)
    feature_engineering.preprocess_data()
   
except Exception as e:
    raise e

[2023-07-11 17:12:52,237: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-11 17:12:52,246: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-11 17:12:52,249: INFO: common: created directory at: artifacts]
[2023-07-11 17:12:52,252: INFO: common: created directory at: artifacts/data_ingestion/Brain_MRI]
The training and testing CSV files have been saved.
